# 基于PointNet++的点云分类和语义分割实验
随着卷积神经网络 CNN 的快速发展,CNN 在二维图像领域取得了十分优秀的成果。但推广到三维领域,无论是成倍增加的网络体积和计算量,还是数据存储量,都是影响卷积神经网络在三维领域发展的障碍。与二维图像上的检测分割不同的是,3D 检测的任务主要是确定可以表示某一种类目标姿态的 3D 边界框,它即包含目标的空间位置信息,也包含目标的朝向、旋转状态等信息;3D分割的任务主要是分割点云,区分不同种类的物体,将整个点云划分为各种在语义上有意义的部分或是各个有意义的个体。

点云数据是一个由无序的数据点构成的集合,因此点云数据上的深度学习一直是一个较为困难的任务。在使用深度学习模型处理点云数据之前,往往需要对点云数据进行处理。



## PointNet
PointNet 是 Qi 等人在 2017 年 CVPR 会议的一篇文章中提出的方法,其主要完成 3D 检测和分割
PointNet是Qi等人在2017年CVPR会议的一篇文章中提出的方法，其主要完成3D检测和分割任务。PointNet提出直接在点云数据上应用深度学习模型，并且用实验证明其高准确率性能。PointNet主要针对点云数据的无序性和空间变换的不变性进行论证，并提出了很好的解决方法。文章详情见PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation。
无序性指点云数据是由可任意排列的数据组成的集合，而要使用深度学习模型的一个前提是需要保证不论点云顺序如何，都应该提取得到相同的特征。在PointNet中作者提出用Maxpooling对称函数来提取特征，解决无序性问题。Maxpooling即在每一维的特征都选取N个点中对应的最大的特征值。
变换不变性是指点云数据所表示的目标经过一定的空间变换（旋转、平移等）后应该保持不变，在坐标系中即为点云数据坐标发生变化后，不论其用何种的坐标系表示，网络都能正确地识别目标。在PointNet中作者提出在对点云数据提取特征前先用STN（Spatial Transform Network）对齐以保证其不变性。STN如图6-27中两转换网络示意图所示，T-Net为了让特征对齐，会通过学习点云的位置信息来找到最适合的旋转角度，得到一个转换矩阵，并通过将其与输入点云数据相乘来保证数据的不变性。
PointNet的网络结构如下图所示，其主要流程为：

1. 输入数据为（为点云个数，对应点云的空间坐标）的点云数据集合，并将其表示为的张量。
2. 首先将输入数据通过一个STN，其将T-Net学习到的转换矩阵与输入数据相乘来进行对齐以保证其对一定的空间变换保持不变性。
3. 通过多层感知机来提取点云数据特征，将其特征维数升到64维后，再通过一个STN对齐特征。
4. 再次通过多层感知机MLP提取点云数据特征，将其特征维数升到1024维后，使用Maxpooling得到其全局特征。
5. 若目的为分类，则直接将全局特征通过多层感知机得到各个种类的预测分数；若目的为分割，则将全局特征与之前提取到的64维特征（局部特征）进行串联，在通过多层感知机对每个点进行分类。

 ![PointNet网络结构图](images/PointNet.png)
 
PointNet++是Charles发表在机器学习会议NIPS（2017）的一篇文章，其思想基于改进PointNet局部特征提取变现不好的缺点，提出了一种分层神经网络结构（Hierarchical Neural Nettwork），这种分层结构有一系列的set abstraction组成，如图6-28所示，其包括采样层（Sampling layer）、组合层（Grouping layer）和特征提取层（PointNet layer）。PointNet++在不同尺度下提取特征作为局部特征，并通过多层网络结构得到深层特征。文章详情见PointNet++: Deep Hierarchical Feature Learning on Point Sets in a Metric Space。

![PointNet++网络结构图](images/PointNet++.png)

## 1. 实验环境
本实验在erikwijmans的PointNet2_PyTorch上改进而来。
### 1.0 Python = 3.5
本实验使用Python 3.5及其以上版本的Python
### 1.1 Pytorch = 0.4.1
本实验使用Pytorch 0.4.1以上版本，读者需要根据自己计算机的设备选择合适的CUDA版本，或CPU版本
### 1.2 Visodm
本实验需要使用Visdom可视化工具，可实时监控训练进程，更加直观的查看训练过程中准确度和损失函数的变化，从而在整体上判断训练过程是否过拟合等。
Visdom是一种可视化工具，支持Pytorch和numpy，可以实现远程数据的实时的可视化，清楚的显示各个数据随时间的变化，如图6-31所示，为本次实验训练PointNet++分割模型过程中的可视化界面。Visdom界面如下图所示：
![visdom](images/visdom.png)

通过以下命令来安装visdom:
```shell
pip3 install visdom
```

安装好visdom之后，在命令行终端输入```python -m visdom.server```来开启端口：

![vidom端口](images/visdom端口.png)

在本实验中，由于实验环境部署在docker中，因此，需要将```8097```端口映射到宿主机的端口上，才能在浏览器中实时显示训练过程的loss变换。本实验中对visdom类的定义在```pointnet2/utils/pytoch_utils/visdomvis.py```中。

## 2. Semantic-Kitti数据集
本实验使用著名的Kitti数据集来作为激光点云的分割数据集。Semantic-Kitti是基于KITTI Vision Benchmark提出的一个大规模激光点云语义分割和全景分割数据集，Semantic-Kitti的作者为KITTI的00-10序列进行了密集的语义标注，使得可以使用多个顺序扫描进行语义场景解释，例如语义分段和语义场景完成。而KITTI的其余的序列，即序列11-21，则被用作测试集，显示了各种具有挑战性的交通状况和环境类型。

![训练集](images/overview-train.svg)

![测试集](images/overview-test.svg)

Semantic-Kitti中包含28类不同的语义标签，包括了静止目标和动态目标，其标签分布如下所示：
![标签分布](images/label_distribution.svg)

## 3. 网络结构代码

本部分将使用pytorch来搭建网络

In [1]:
import torch
import torch.nn as nn

网络结构如下图所示：

![网络结构](images/PointNet++.png)

### 3.1  Set Abstraction层
PointNet++中，基础网络层是由若干个Set Abstraction层构成。这些曾简称SA层，由一个Sampling Layer、Grouping Layer和一个Pointnet组件构成。为了方便搭建和修改网络，我们需要先构建SA层。

下面介绍一下Set Abstraction的具体组成：
* 采样层：即在处理点云数据之前，先进行采样处理。在该文章中，Charles使用迭代最远点采样（Farthest Point Sampling，FPS）方法。该方法根据选定的一个点云数据子集，对子集中每个点抽取其距离最远的点，得到一个新的子集，这两个子集的并集就是样本空间。与随机取样相比，FPS能更好的覆盖整个点云数据。采样层的输入为$N\times(d + C)$，其中d为点云的坐标，C为特征维度，N为输入点数量
* 组合层：其根据采样点定义了一个局部域，方便下一步在该区域中提取特征。其输入为的点云集合以及个质心的坐标，输出为的点云集合，其中是局部区域除了质心点之外的点数。在卷积神经网络中，一张图像像素的局部区域指像素周围曼哈顿距离下的领域点，而点云中一个点的局部区域指该点几何距离下的邻域点组成。与临近算法不同的是，该文章用了Ball Query方法，其选取固定半径区域内的点（设有上限）。
* 特征提取层：即使用PointNet对组合层中的局部区域提取特征。在PointNet++代码中，这一部分进行了化简，使用了一个max_pool2d来作为特征提取层。

另外，不同于图像中各个像素点密度均匀分布，点云数据在空间上分布密度不均匀、不规则，在距离较远的地方激光点云十分稀疏。无差别的提取特征会使得网络很容易在低密度的地方丢失局部信息，而导致PointNet训练效果不理想，造成较大的误差。因此，在点云密度较小的地方应该加大选取的局部区域来更好的提取特征。Charles提出密度自适应PointNet层，来组合不同密度点云数据的特征。文章中提出了两种组合的方式：多尺度组合（Multi-scale Grouping, MSG）和多分辨率组合（Multi-resolution Grouping, MRG）。

In [2]:
# SA层的基础结构
class _PointnetSAModuleBase(nn.Module):

    def __init__(self):
        super().__init__()
        self.npoint = None
        self.groupers = None
        self.mlps = None
        
    def forward(self, xyz: torch.Tensor,features: torch.Tensor = None) -> (torch.Tensor, torch.Tensor):
        r"""
        Parameters
        xyz : torch.Tensor
                    输入维度(B, N, 3)，数据的三维特征表示
        features : torch.Tensor 
                    输入维度(B, N, C)，特征表示,B 为batchsize，N为点云数目，3为三维坐标
        xyz与feature共同构成了SA层的输入
        Returns
        new_xyz : torch.Tensor
                    输出维度(B, npoint, 3)，分组点云的三维表示
        new_features : torch.Tensor(B, npoint, \sum_k(mlps[k][-1]))，每组点云的特征表示
        new_xyz与new_features共同构成了SA层的输出
        """
        new_features_list = []
        # 对点云进行下采样
        xyz_flipped = xyz.transpose(1, 2).contiguous()
        new_xyz = pointnet2_utils.gather_operation(
            xyz_flipped,
            pointnet2_utils.furthest_point_sample(xyz, self.npoint)
        ).transpose(1, 2).contiguous() if self.npoint is not None else None
        # 计算经过最远点采样和分组处理之后的新点云数据
        for i in range(len(self.groupers)):
            # groupper层的
            new_features = self.groupers[i](
                xyz, new_xyz, features)  
            # (B, C, npoint, nsample)
            new_features = self.mlps[i](new_features)  
            # (B, mlp[-1], npoint, nsample)
            new_features = F.max_pool2d(new_features, kernel_size=[1, new_features.size(3)]) 
            # (B, mlp[-1], npoint, 1)
            new_features = new_features.squeeze(-1) 
		 # (B, mlp[-1], npoint)
            new_features_list.append(new_features)
        #提取特征过程中先分组，对每一分组，用多层感知机提取特征，最后使用全局最大池化提取全局特征。
        return new_xyz, torch.cat(new_features_list, dim=1)
        #返回分组后的数据以及存有每组特征的列表

述代码中，使用了pointnet2_utils中定义的gather_operation和furthest_point_sample方法，来实现对点云的下采样。

在定义好_PointNetSAModuleBase类之后，接下来就要正式定义SA层了。PointNet++的SA层分为两种，一种是基于多尺度Grouping的SA层，一种是基于单尺度Grouping的SA层。二者的定义如下所示：

In [3]:
from typing import List

# 多尺度Grouping的SA层
class PointnetSAModuleMSG(_PointnetSAModuleBase):
    r"""Pointnet set abstrction layer with multiscale grouping

    Parameters
    ----------
    npoint : int
        Number of features
    radii : list of float32
        list of radii to group with
    nsamples : list of int32
        Number of samples in each ball query
    mlps : list of list of int32
        Spec of the pointnet before the global max_pool for each scale
    bn : bool
        Use batchnorm
    """

    def __init__(self,*,npoint: int,radii: List[float],nsamples: List[int], mlps: List[List[int]], bn: bool = True,  use_xyz: bool = True):
        super().__init__()

        assert len(radii) == len(nsamples) == len(mlps)

        self.npoint = npoint
        self.groupers = nn.ModuleList()
        self.mlps = nn.ModuleList()
        for i in range(len(radii)):
            radius = radii[i]
            nsample = nsamples[i]
            self.groupers.append(
                pointnet2_utils.QueryAndGroup(radius, nsample, use_xyz=use_xyz)
                if npoint is not None else pointnet2_utils.GroupAll(use_xyz)
            )
            mlp_spec = mlps[i]
            if use_xyz:
                mlp_spec[0] += 3

            self.mlps.append(pt_utils.SharedMLP(mlp_spec, bn=bn))

In [4]:
# 单尺度Grouping的SA层，直接继承了PointnetSAModuleMSG层
class PointnetSAModule(PointnetSAModuleMSG):
    r"""Pointnet set abstrction layer

    Parameters
    ----------
    npoint : int
        Number of features
    radius : float
        Radius of ball
    nsample : int
        Number of samples in the ball query
    mlp : list
        Spec of the pointnet before the global max_pool
    bn : bool
        Use batchnorm
    """

    def __init__(
            self,
            *,
            mlp: List[int],
            npoint: int = None,
            radius: float = None,
            nsample: int = None,
            bn: bool = True,
            use_xyz: bool = True
    ):
        super().__init__(
            mlps=[mlp],
            npoint=npoint,
            radii=[radius],
            nsamples=[nsample],
            bn=bn,
            use_xyz=use_xyz
        )

### 3.2 Segmentation层
分割任务中，SA层之后，还需要接一个Segmentation网络，以实现分割。Segementation通过skip line connection操作不断与底层的低阶信息融合，最终实现对原数据中每一个点云的语义分割结果。

由于在SA层中，对原始点云进行了若干次下采样。因此，需要在Segmentation网络中进行上采样，以还原原始数据。PointNet++中采用基于距离的插值和across level skip links的分层传播策略来将分割结果传播到原始点云数据中。在特征传播级别中，作者将点特征从$N_l \times (d + C)$个点传播到个$N_{l - 1}$个点，其中$N_{l-1}$和$N_l$分别为第$l$级SA层的输入和输出的点云的数量。

Segementation网络中，基本网路单元定义为FP层：

In [5]:
class PointnetFPModule(nn.Module):
    r"""Propigates the features of one set to another

    Parameters
    ----------
    mlp : list
        Pointnet module parameters
    bn : bool
        Use batchnorm
    """

    def __init__(self, *, mlp: List[int], bn: bool = True):
        super().__init__()
        self.mlp = pt_utils.SharedMLP(mlp, bn=bn)

    def forward(
            self, unknown: torch.Tensor, known: torch.Tensor,
            unknow_feats: torch.Tensor, known_feats: torch.Tensor
    ) -> torch.Tensor:
        r"""
        Parameters
        ----------
        unknown : torch.Tensor
            (B, n, 3) tensor of the xyz positions of the unknown features
        known : torch.Tensor
            (B, m, 3) tensor of the xyz positions of the known features
        unknow_feats : torch.Tensor
            (B, C1, n) tensor of the features to be propigated to
        known_feats : torch.Tensor
            (B, C2, m) tensor of features to be propigated

        Returns
        -------
        new_features : torch.Tensor
            (B, mlp[-1], n) tensor of the features of the unknown features
        """

        if known is not None:
            dist, idx = pointnet2_utils.three_nn(unknown, known)
            dist_recip = 1.0 / (dist + 1e-8)
            norm = torch.sum(dist_recip, dim=2, keepdim=True)
            weight = dist_recip / norm

            interpolated_feats = pointnet2_utils.three_interpolate(
                known_feats, idx, weight
            )
        else:
            interpolated_feats = known_feats.expand(
                *known_feats.size()[0:2], unknown.size(1)
            )

        if unknow_feats is not None:
            new_features = torch.cat([interpolated_feats, unknow_feats],
                                   dim=1)  #(B, C2 + C1, n)
        else:
            new_features = interpolated_feats

        new_features = new_features.unsqueeze(-1)
        new_features = self.mlp(new_features)

        return new_features.squeeze(-1)

### 3.3 完整的PointNet++
至此，我们已经完成PointNet++网络中的结构单元的定义，截下来可以定义一个PointNet++网络了。

根据不同组合，PointNet++有四种不同网络结构：多尺度策略的分类和分割网络、单尺度的分类和分割网络。我们以多尺度策略的分割网络为例，来构建一个PointNet++网络。

In [6]:
# pointnet2_msg_sem网络结构

class Pointnet2MSG(nn.Module):
    r"""
        PointNet2 with multi-scale grouping
        Semantic segmentation network that uses feature propogation layers

        Parameters
        ----------
        num_classes: int
            Number of semantics classes to predict over -- size of softmax classifier that run for each point
        input_channels: int = 6
            Number of input channels in the feature descriptor for each point.  If the point cloud is Nx9, this
            value should be 6 as in an Nx9 point cloud, 3 of the channels are xyz, and 6 are feature descriptors
        use_xyz: bool = True
            Whether or not to use the xyz position of a point as a feature
    """

    def __init__(self, num_classes, input_channels=6, use_xyz=True):
        super().__init__()

        self.SA_modules = nn.ModuleList()
        
        # 1st SA Layer
        c_in = input_channels
        self.SA_modules.append(
            PointnetSAModuleMSG(
                npoint=1024,
                radii=[0.05, 0.1],
                nsamples=[16, 32],
                mlps=[[c_in, 16, 16, 32], [c_in, 32, 32, 64]],
                use_xyz=use_xyz
            )
        )
        c_out_0 = 32 + 64
        
        # 2nd SA Layer
        c_in = c_out_0
        self.SA_modules.append(
            PointnetSAModuleMSG(
                npoint=256,
                radii=[0.1, 0.2],
                nsamples=[16, 32],
                mlps=[[c_in, 64, 64, 128], [c_in, 64, 96, 128]],
                use_xyz=use_xyz
            )
        )
        c_out_1 = 128 + 128
        
        # 3rn SA Layer
        c_in = c_out_1
        self.SA_modules.append(
            PointnetSAModuleMSG(
                npoint=64,
                radii=[0.2, 0.4],
                nsamples=[16, 32],
                mlps=[[c_in, 128, 196, 256], [c_in, 128, 196, 256]],
                use_xyz=use_xyz
            )
        )
        c_out_2 = 256 + 256
        
        # 4th SA Layer
        c_in = c_out_2
        self.SA_modules.append(
            PointnetSAModuleMSG(
                npoint=16,
                radii=[0.4, 0.8],
                nsamples=[16, 32],
                mlps=[[c_in, 256, 256, 512], [c_in, 256, 384, 512]],
                use_xyz=use_xyz
            )
        )
        c_out_3 = 512 + 512

        self.FP_modules = nn.ModuleList()
        # 1st FP Layer
        self.FP_modules.append( PointnetFPModule(mlp=[256 + input_channels, 128, 128]))
        # 2nd FP Layer
        self.FP_modules.append(PointnetFPModule(mlp=[512 + c_out_0, 256, 256]))
        # 3rd FP Layer
        self.FP_modules.append(PointnetFPModule(mlp=[512 + c_out_1, 512, 512]))
        # 4th FP Layer
        self.FP_modules.append( PointnetFPModule(mlp=[c_out_3 + c_out_2, 512, 512]))
        
        # 全连接层
        self.FC_layer = nn.Sequential(
            pt_utils.Conv1d(128, 128, bn=True), nn.Dropout(),
            pt_utils.Conv1d(128, num_classes, activation=None)
        )
    
    def _break_up_pc(self, pc):
        xyz = pc[..., 0:3].contiguous()
        features = (
            pc[..., 3:].transpose(1, 2).contiguous()
            if pc.size(-1) > 3 else None
        )

        return xyz, features

    def forward(self, pointcloud: torch.cuda.FloatTensor):
        r"""
            Forward pass of the network

            Parameters
            ----------
            pointcloud: Variable(torch.cuda.FloatTensor)
                (B, N, 3 + input_channels) tensor
                Point cloud to run predicts on
                Each point in the point-cloud MUST
                be formated as (x, y, z, features...)
        """
        xyz, features = self._break_up_pc(pointcloud)

        l_xyz, l_features = [xyz], [features]
        # pdb.set_trace()
        for i in range(len(self.SA_modules)):
            li_xyz, li_features = self.SA_modules[i](l_xyz[i], l_features[i])
            l_xyz.append(li_xyz)
            l_features.append(li_features)

        for i in range(-1, -(len(self.FP_modules) + 1), -1):
            l_features[i - 1] = self.FP_modules[i](
                l_xyz[i - 1], l_xyz[i], l_features[i - 1], l_features[i]
            )

        return self.FC_layer(l_features[0]).transpose(1, 2).contiguous()


## 4.Semantic-Kitti数据集的分割任务实验
本部分使用PointNet++，在Kitti数据集上进行分割实验。

本实验中，选择序列00的前100帧激光点云来进行实验。本实验的数据位于```dataset```文件夹下，其结构如下所示：

![数据文件夹结构](images/folder_structure.svg)

```0000xx.bin```文件为点云文件，```0000xx.label```为点云中每个点对应的标签。我们使用jbehley等人所提供的工具来进行数据的读写(https://github.com/PRBonn/semantic-kitti-api )，这个API在```Pointnet2_PyTorch-master/pointnet2/data/kitti_api```中。

基于上述的API，我们构建了一个```torch.data.Dataset```类来负责训练和测试时的数据读取：

In [7]:
import os
import torch.utils.data as data
import numpy as np
import glob
import yaml


from pointnet2.data.kitti_api.auxiliary.laserscan import *

class Kitti3DSemSeg(data.Dataset):
    def __init__(self, num_points, train = True, download = True, data_precent = 1.0):
        super().__init__()
        self.data_precent = data_precent
        self.train, self.num_points = train, num_points

        pcd_path = '/kitti_semantic/dataset/sequences/00/velodyne/'
        label_path = '/kitti_semantic/dataset/sequences/00/labels/'
        pcd_list, label_list = [], []
        points, labels = [], []
        # 写入pcd_path的路径
        pcd_list = glob.glob(os.path.join(pcd_path, '*.bin'))
        # 写入label_path的路径
        label_list = glob.glob(os.path.join(label_path, '*.label'))
        # 将文件list重排序
        pcd_list.sort()
        label_list.sort()

        # 验证pcd_list与label_list的长度是否相同
        assert(len(pcd_list) == len(label_list))
        num_of_file = len(pcd_list)

        # 读取点云及其标签
        config_path = '/kitti_semantic/Pointnet2_PyTorch-master/pointnet2/data/kitti_api/config/semantic-kitti.yaml'
        CFG = yaml.safe_load(open(config_path, 'r'))
        color_dict = CFG["color_map"]
        class_remap = CFG["learning_map"]

        nclasses = len(color_dict)
        print(nclasses)
        sem_ls = SemLaserScan(nclasses, color_dict, project = False)

        k = 0
        for i in range(0, num_of_file):
            scan_file = pcd_list[i]
            label_file = label_list[i]
            
            sem_ls.open_scan(scan_file)
            sem_ls.open_label(label_file)
            scans = sem_ls.points.tolist()
            sems = sem_ls.sem_label.astype(np.float32).tolist()
            # 每片点云中选择100592个点
            if len(scans) >= 100592 and len(sems) >= 100592:
                points.extend(scans[0 : 100592])
                labels.extend(sems[0 : 100592])
                k += 1
        
        points = np.array(points)
        print(points.shape)
        points = np.reshape(points, (k, 100592, 3))
        self.points = np.tile(points, (1, 1, 3))
        labels_remap = []
        for x in labels:
            labels_remap.append(class_remap[x])
        labels = np.array(labels_remap)
        print(len(labels.nonzero()[0]))
        self.labels = np.reshape(labels, (num_of_file, 100592))

        print(self.points.shape)
        print(self.labels.shape)
        print(max(labels))

    def __getitem__(self, idx):
        pt_idxs = np.arange(0, self.num_points)
        # np.random.shuffle(pt_idxs)

        current_points = torch.from_numpy(self.points[idx, pt_idxs].copy()
                                         ).type(torch.FloatTensor)
        # pdb.set_trace()
        current_labels = torch.from_numpy(self.labels[idx, pt_idxs].copy()
                                         ).type(torch.LongTensor)

        return current_points, current_labels

    def __len__(self):
        return int(self.points.shape[0] * self.data_precent)

    def set_num_points(self, pts):
        self.num_points = pts

    def randomize(self):
        pass
    

# 测试数据读取
dset = Kitti3DSemSeg(16, train = True)
print(dset[0])
print(len(dset))
dloader = torch.utils.data.DataLoader(dset, batch_size=32, shuffle=True)
for i, data in enumerate(dloader, 0):
    inputs, labels = data
    if i == len(dloader) - 1:
        print(inputs.size())

34
(10059200, 3)
9799219
(100, 100592, 9)
(100, 100592)
19
(tensor([[52.8979,  0.0230,  1.9980, 52.8979,  0.0230,  1.9980, 52.8979,  0.0230,
          1.9980],
        [53.7505,  0.1929,  2.0270, 53.7505,  0.1929,  2.0270, 53.7505,  0.1929,
          2.0270],
        [53.8031,  0.3618,  2.0289, 53.8031,  0.3618,  2.0289, 53.8031,  0.3618,
          2.0289],
        [72.6007,  1.2965,  2.6647, 72.6007,  1.2965,  2.6647, 72.6007,  1.2965,
          2.6647],
        [72.1183,  1.5134,  2.6477, 72.1183,  1.5134,  2.6477, 72.1183,  1.5134,
          2.6477],
        [74.4768,  1.7983,  2.7276, 74.4768,  1.7983,  2.7276, 74.4768,  1.7983,
          2.7276],
        [73.2674,  1.9992,  2.6876, 73.2674,  1.9992,  2.6876, 73.2674,  1.9992,
          2.6876],
        [72.1100,  2.1931,  2.6485, 72.1100,  2.1931,  2.6485, 72.1100,  2.1931,
          2.6485],
        [72.9786,  2.4500,  2.6775, 72.9786,  2.4500,  2.6775, 72.9786,  2.4500,
          2.6775],
        [72.9362,  2.6779,  2.6764, 72.9

需要注意的是，原本的```.label```中存储的label并不能直接用于训练，我们需要通过```semantic-kitti.yaml```文件中的learning_map来得到映射，将其转换为能够直接用于训练的label。反之，由于semantic-kitti-api的可视化工具中需要使用原始的label，因此，在后续的测试中，我们需要将训练得到的label通过learning_map_inv来映射为原始的label。

## 5. 训练
在构建好数据读取、定义好模型之后，可以开始训练了。为了方便实验，PointNet2_PyTorch原作者构建和使用了多个pytorch的工具，用来便捷地开始训练和测试。这里我们也继承了他的方法。

In [8]:
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_sched
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
import numpy as np
import os
import tqdm

# 添加模型，上面介绍的模型结构在文件夹models下有对应的.py文件
from pointnet2.models import Pointnet2SemMSG as Pointnet
from pointnet2.models.pointnet2_msg_sem import model_fn_decorator
from pointnet2.data.KittiLoader import Kitti3DSemSeg
import pointnet2.utils.pytorch_utils as pt_utils

### 5.1 参数设置

In [9]:
# batch大小
batch_size = 8
# 每帧点云中有多少点
num_points = 100592
# L2正则化系数
weight_decay = 0
# 学习率
lr = 1e-3
# 学习率衰减指数
lr_decay = 0.5
# 学习率衰减step
decay_step = 2e3
# batch norm monentum
bn_momentum = 0.9
# batch norm momentum decay gamma
bn_decay = 0.5
# epoch
epoches = 600

### 5.2 数据读取

In [10]:
# 测试集
test_set = Kitti3DSemSeg(num_points, train=False)
test_loader = DataLoader(test_set, batch_size=batch_size,shuffle=True,pin_memory=True,num_workers=0)

34
(10059200, 3)
9799219
(100, 100592, 9)
(100, 100592)
19


In [11]:
# 训练集
train_set = Kitti3DSemSeg(num_points)
train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=0,
    shuffle=True
)

34
(10059200, 3)
9799219
(100, 100592, 9)
(100, 100592)
19


### 5.3 开始训练

In [12]:
# 导入模型
model = Pointnet(num_classes=34, input_channels=6, use_xyz=True)
_ = model.cuda()

In [13]:
lr_clip = 1e-5
bnm_clip = 1e-2

# 设置optimizer
optimizer = optim.Adam(
    model.parameters(), 
    lr=lr, 
    weight_decay=weight_decay
)

# 设置lr_lbmd和bnm_lmdd
lr_lbmd = lambda it: max(lr_decay**(int(it * batch_size / decay_step)), lr_clip / lr)
bnm_lmbd = lambda it: max(bn_momentum * bn_decay**(int(it * batch_size / decay_step)), bnm_clip)

# 设置训练计划
lr_scheduler = lr_sched.LambdaLR(optimizer, lr_lbmd)
bnm_scheduler = pt_utils.BNMomentumScheduler(model, bnm_lmbd)
start_epoch = 1
best_prec = 0
best_loss = 1e10

# 设置全连接层
model_fn = model_fn_decorator(nn.CrossEntropyLoss())

# 设置viz
# viz = pt_utils.VisdomViz(port=8097)
# viz.text(str('PointNet2'))

In [14]:
# 开始训练
trainer = pt_utils.Trainer(
    model,
    model_fn,
    optimizer,
    checkpoint_name="/kitti_semantic/Pointnet2_PyTorch-master/pointnet2/train/checkpoints/pointnet2_smeseg",
    best_name="/kitti_semantic/Pointnet2_PyTorch-master/pointnet2/train/checkpoints/pointnet2_semseg_best",
    lr_scheduler=lr_scheduler,
    bnm_scheduler=bnm_scheduler
    # viz=viz
)

trainer.train(
    0,
    start_epoch,
    epoches,
    train_loader,
    test_loader,
    best_loss=best_loss
)

if start_epoch == epoches:
    _ = trainer.eval_epoch(test_loader)

epochs:   0%|          | 0/600 [00:10<?, ?it/s]39it/s, total_it=13]
                                                                   
val: 100%|██████████| 13/13 [00:06<00:00,  2.09it/s]
                                                    
epochs:   0%|          | 1/600 [00:19<3:15:27, 19.58s/it]al_it=15]


KeyboardInterrupt: 

## 6. 测试
在完成训练之后，我们可以使用数据进行测试了。

### 6.1 设置参数

In [15]:
# 设置预测结果输出文件
preds_path = '/kitti_semantic/dataset/sequences/00/preds1'
# batch_size
batch_size = 1
# num_points
num_points = 100592
# device_name
device_name = 'cuda'

### 6.2 读取测试集数据
为了减轻服务器负担，本实验选择训练集当做测试集，读者也可将Kitti中的别的数据作为测试集进行测试。

In [16]:
test_set = Kitti3DSemSeg(num_points, train = False)
test_loader = DataLoader(test_set, batch_size = batch_size, 
                         shuffle = False, 
                         pin_memory = True,
                         num_workers = 0)

34
(10059200, 3)
9799219
(100, 100592, 9)
(100, 100592)
19


### 6.3 读取模型参数

In [17]:
device = torch.device(device_name)
model = Pointnet(num_classes = 34, input_channels = 6, use_xyz = True)
model.load_state_dict(torch.load(
    '/kitti_semantic/Pointnet2_PyTorch-master/pointnet2/train/pointnet2_semseg_best.pth.tar', 
    map_location = device
)['model_state'])
_ = model.cuda()

### 6.4 进行测试
进行测试，并将测试结果以Semantic-Kitti的.label文件格式存储下来。

In [18]:
# 读取learning_map_inv，用于将测试得到的label转换为可视化工具需要的label
config_path = '/kitti_semantic/Pointnet2_PyTorch-master/pointnet2/data/kitti_api/config/semantic-kitti.yaml'
CFG = yaml.safe_load(open(config_path, 'r'))
class_inv_map = CFG['learning_map_inv']
# ground truth文件夹
label_path = '/kitti_semantic/dataset/sequences/00/labels'

In [19]:
# 对测试数据进行训练
for i,data in enumerate(test_loader):
    f = os.path.join(preds_path, str(i).zfill(6) + '.label')
    f_l = os.path.join(label_path, str(i).zfill(6) + '.label')
    inputs, labels = data
    inputs = inputs.to(device, non_blocking = True)
    labels = labels.to(device, non_blocking = True)
    
    preds = model(inputs)
    _, classes = torch.max(preds, -1)

    acc = (classes == labels).float().sum() / labels.numel()
    print('Frame %d Segmantion Accuracy: %-10.6s' %  (i, acc.item()))

    classes = classes.clone().cpu().numpy()
    classes = classes.reshape(num_points).tolist()
    classes_bin = []
    for i in range(len(classes)):
        classes_bin.append(class_inv_map[classes[i]])

    # 找出与源点云的数量的差值，补齐没有被选作为测试的点，可视化用
    k = len(np.fromfile(f_l,dtype = np.uint32).tolist()) - len(classes)
    for i in range(k):
        classes_bin.append(0)
    
    # 将测试结果写入.label文件中
    classes_bin = np.array(classes_bin, dtype = np.uint32)
   #  print(classes_bin.shape)
    classes_bin.tofile(f)

Frame 0 Segmantion Accuracy: 0.6839    
Frame 1 Segmantion Accuracy: 0.6570    
Frame 2 Segmantion Accuracy: 0.7398    
Frame 3 Segmantion Accuracy: 0.7500    
Frame 4 Segmantion Accuracy: 0.8149    
Frame 5 Segmantion Accuracy: 0.8684    
Frame 6 Segmantion Accuracy: 0.8777    
Frame 7 Segmantion Accuracy: 0.8575    
Frame 8 Segmantion Accuracy: 0.8554    
Frame 9 Segmantion Accuracy: 0.8136    
Frame 10 Segmantion Accuracy: 0.8787    
Frame 11 Segmantion Accuracy: 0.8565    
Frame 12 Segmantion Accuracy: 0.8901    
Frame 13 Segmantion Accuracy: 0.9167    
Frame 14 Segmantion Accuracy: 0.8689    
Frame 15 Segmantion Accuracy: 0.9190    
Frame 16 Segmantion Accuracy: 0.9015    
Frame 17 Segmantion Accuracy: 0.9237    
Frame 18 Segmantion Accuracy: 0.8910    
Frame 19 Segmantion Accuracy: 0.8250    
Frame 20 Segmantion Accuracy: 0.8098    
Frame 21 Segmantion Accuracy: 0.7853    
Frame 22 Segmantion Accuracy: 0.7385    
Frame 23 Segmantion Accuracy: 0.8200    
Frame 24 Segmantion Accura

### 6.5 可视化结果展示： 

我们随机选择了三帧点云来进行可视化，其结果如下：

frame_0:
![frame_0](images/frame_0_pred.png)

frame_51:
![frame_0](images/frame_51_pred.png)

frame_99:
![frame_0](images/frame_99_pred.png)